## Getting started with Dask and Running Jupyter Notebooks on the DoD Utility Servers ## 

Note: this is a modified version of this notebook: https://github.com/pangeo-data/pangeo/wiki/Getting-Started-with-Dask-on-Cheyenne# 

This notebook documents how to set up a virtual Python environment on the Department of Defense Utility Servers. In particular it covers the following:

1) Install conda and create a virtual environment

2) Configure Jupyter

3) Launch Dask with a job scheduler

4) Launch a Jupyter server for your job

5) Connect to Jupyter and the Dask dashboard from your personal computer

This assumes that you are already fairly comfortable using the command line, such as using a text editor. 

To start with, log on to the Utility Server you plan to use (AFRL, ERDC, NAVY). 

## Installing a software environment ##

After you have logged into the Utility Server of your choice (AFRL, ERDC, Navy), download and install Miniconda. 

In [ ]:
wget https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
chmod +x Miniconda3-latest-Linux-x86_64.sh
./Miniconda3-latest-Linux-x86_64.sh

This process will take a few minutes. 

This contains a self-contained Python environment that we can manipulate safely without requiring the involvement of IT. It also allows you to create isolated software environments so that we can experiment in the future safely.

In [ ]:
conda create -n pangeo -c conda-forge \
    python=3.6 dask distributed xarray jupyterlab mpi4py

This will also take a few minutes - it's a good time to get a cup of coffee. 

After creating the environment, you will need to either log out and log back in or start a new terminal for your `conda` commands to work on the command line. 

Activate this environment

In [ ]:
source activate pangeo

Your prompt should now look like this:

In [ ]:
(pangeo) [gergel@viutill-0002 ~]$

And if you ask where your Python command lives, it should direct you to somewhere in your home directory:

In [ ]:
(pangeo) [gergel@viutill-0002 ~]$ which python
~/miniconda3/envs/pangeo/bin/python

## Configure Jupyter to use Jupyter Notebooks ##

Note: if you only want to run python scripts on the Utility Servers, you can skip all of the following sections and just activate the python environment you created to run Python scripts. At any point in time, if you need to add a Python package, you can do so using `conda install package_name` after activating the environment on the command line. 

Jupyter notebook servers include a password for security. We're going to setup a password for ourselves. First we generate the Jupyter config file

In [ ]:
jupyter notebook --generate-config

Your output should look like the following: 

In [ ]:
Writing default config to: /u/US_HOME2/gergel/.jupyter/jupyter_notebook_config.py

If you open that file and search for "password", you'll see a line like the following:

In [ ]:
#c.NotebookApp.password = ''

The instructions in the comments of the config file tell you to generate a hashed password by entering the following commands:

In [ ]:
ipython

In [ ]:
from notebook.auth import passwd; passwd()
Enter password:

You can enter a password of your choice, and it will return to you a hashed password that encodes the same information, but is safe to include in a publicly accessible config file. I got the following output:

In [ ]:
Out[1]: 'sha1:30adc662b899:24963c5aae5ac22f0f433cfbe22d590dd7054db5'

After exiting from Ipython, copy that string into your jupyter_notebook_config.py config file and save the config file: 

In [ ]:
c.NotebookApp.password = u'sha1:69a76df803b9:99ca27341563cd85ba4e78684128e1f4ad2d8d0d'

## Launch Dask with a script

Copy and paste the following text into a file and name the file dask.sh:

In [ ]:
#!/bin/bash
#PBS -N dask
#PBS -q parallel
#PBS -A NPSCA07935YF5
#PBS -l select=1:ncpus=32:mpiprocs=8:mem=200GB
#PBS -l walltime=12:00:00
#PBS -j oe
#PBS -m abe

# Qsub template for AFRL US
# Scheduler: PBS

# This writes a scheduler.json file into your home directory
# You can then connect with the following Python code
# >>> from dask.distributed import Client
# >>> client = Client(scheduler_file='~/scheduler.json')

export LANG="en_US.utf8"
export LANGUAGE="en_US.utf8"
export LC_ALL="en_US.utf8"

unset LD_LIBRARY_PATH
source activate pangeo

rm -f scheduler.json
mpirun --np 8 dask-mpi --nthreads 4 --memory-limit 24e9 # --interface ib0

This script asks for one nodes with 32 cores each. It breaks up each node into 8 MPI processes. You can tweak the numbers above if you like, but you'll have to match some constraints in the PBS directives on the top and the mpirun keywords on the bottom.

Submit this script to run on the cluster with `qsub`

In [ ]:
qsub dask.sh

Once you submit the job you'll get a job ID: 

In [ ]:
116716.viutill-0003.erdc.hpc.mil

And you can check on the status of the job: 

In [ ]:
qstat -u $USER

When this job runs it places a `scheduler.json` file in your home directory. This contains the necessaary information to connect to this cluster from anywhere in the network. We'll do that now briefly from the login node. In the next section we'll set up a Jupyter notebook server on your allocation.

NOTE: if you *don't* have a `scheduler.json` file in your home directory, the next steps will not work. 

Open up ipython again: 

In [ ]:
ipython

In [ ]:
from dask.distributed import Client
client = Client(scheduler_file='scheduler.json')
client

## Launch and connect to Jupyter ##

From your same session on the login node, run the following code:

In [ ]:
ipython

In [ ]:
from dask.distributed import Client
client = Client(scheduler_file='scheduler.json')

import socket
host = client.run_on_scheduler(socket.gethostname)

def start_jlab(dask_scheduler):
    import subprocess
    proc = subprocess.Popen(['jupyter', 'lab', '--ip', host, '--no-browser'])
    dask_scheduler.jlab_proc = proc

client.run_on_scheduler(start_jlab)

print("ssh -N -L 8787:%s:8787 -L 7777:%s:8888 -l gergel us.erdc.hpc.mil" % (host, host))

NOTE: if you are not familiar with using Ipython, the best way to do the previous step is to go into `ipython`, then type `%cpaste` at the prompt - this will allow you to copy the above chunk of code, paste it in as you would normally, then "enter", then type `--` to signal that you are done pasting in code. 

The above code should print out a statement that looks like this (this is from the Navy Utility Server because the ERDC one was down): 

In [ ]:
ssh -N -L 8787:stutilm-0001.navo.hpc.mil:8787 -L 7777:stutilm-0001.navo.hpc.mil:8888 -l gergel us.navo.hpc.mil

You can run this command from your personal computer (not the terminal logged into Cheyenne) to set up SSH-tunnels that will allow you to log into web servers (e.g. Jupyter notebooks) running on your allocation. Afterwards, you should be able to open the following links in your web browser on your computer. 

So, to connect, open up a new terminal window and type that command at the prompt. This should take you to your Utility Server directory in which you launched the job. If not, then your `dask.sh` script may have timed out, or you may need to repeat the above steps (if the connection is slow, sometimes you may need to go through the steps again). 

Then, go to your browser (I think Chrome works best for this) and type: `localhost:7777/tree`. This should give you a password prompt, enter the same password that you set up at the `Enter password:` step above, and you should be on. **IF** that doesn't work, and your password is invalid, then what you need to do is reset the password. I think this issue may be due to a recent change in Jupyter notebook settings. In this case, in a terminal on the Utility Server after activating your virtual environment, type `jupyter notebook password` and follow the prompt to set the password. This will write the password to a new file, titled `/Users/you/.jupyter/jupyter_notebook_config.json`. You need to *delete* the previous hashed password file, `/Users/you/.jupyter/jupyter_notebook_config.py`. This amounts to a password reset, so you then need to delete your job if it's still running, quit your `ipython` notebook session, and then go through the same steps above to launch dask and then connect to it in `ipython`. This should fix the problem (it did for me when I encountered it).  

Note: the above steps should work on the AFRL, ERDC and NAVY Utility Servers, as long as you update the logins and go through the same steps on both. I have verified them on the Navy and AFRL machines. 